# This Utilizes Tkinter for consumer GUI

In [12]:
# GUI Implement -- Tkinter SQL Converter (with added simple, robust flagging)
# Imports
import sys
import tkinter as tk
from tkinter import filedialog, messagebox

from components.ui_controller import SQLConverterUI
from components.sql_parser import SQLConverter
from components.data_cleaner import SQLCleaner
from components.file_handler import FileHandler

ModuleNotFoundError: No module named 'components'

In [8]:
# Flagging Engine
from dataclasses import dataclass
from typing import Optional, Tuple, List, Dict, Any
import re, difflib

@dataclass (frozen=True)
class SimpleFlag:
    """
    A minimal, UI-friendly flag structure.
    - severity: "info" | "warning" | "error"
    - message: short description
    - source_span: (start_line, end_line) in Tableau SQL
    - target_span: (start_line, end_line) in Fabric SQL
    - code/hint: optional fields for more detail
    """
    severity: str
    message: str
    source_span: Optional[Tuple[int, int]]
    target_span: Optional[Tuple[int, int]]
    code: Optional[str] = None
    hint: Optional[str] = None


#  Tolerate line wrapping / whitespace differences

def _strip_comments(sql: str) -> str:
    """Remove /* ... */ and -- ... comments."""
    sql = re.sub(r'/\*.*?\*/', '', sql, flags=re.DotAll)     # block comments
    sql = re.sub(r'--.*?$', '', sql, flags=re.MULTILINE)     # line comments
    return sql

def _normalize(sql: str) -> str:
    """Collapse all whitespace and remove comments."""
    sql = _strip_comments(sql)
    return re.sub(r'\s+', ' ', sql).strip()

def _split_statements(sql: str) -> List[Dict[str, Any]]:
    """
    Split SQL by semicolons *outside* quotes; return text with line ranges.
    This keeps track of start/end line numbers so UI can show ranges.
    """
    lines = sql.splitlines()
    stmts = []
    buf = []
    start_line = 1

    in_single = False
    in_double = False

    def ends_here(line: str) -> bool:
        nonlocal in_single, in_double
        for ch in line:
            if ch == "'" and not in_double:
                in_single = not in_single
            elif ch == '"' and not in_single:
                in_double = not in_double
        # semicolon ends a statement only if we're not inside quotes
        return (';' in line) and (not in_single) and (not in_double)

    for i, line in enumerate(lines, start=1):
        buf.append(line)
        if ends_here(line):
            text = '\n'.join(buf).strip()
            if text:
                stmts.append({'text': text, 'start': start_line, 'end': i})
            buf = []
            start_line = i + 1

    # trailing part (no semicolon)
    if buf:
        text = '\n'.join(buf).strip()
        if text:
            stmts.append({'text': text, 'start': start_line, 'end': len(lines)})

    return stmts

def _best_match(src: str, cands_norm: List[str]) -> int:
    """Return index of best fuzzy match in normalized candidates (or -1)."""
    if not cands_norm:
        return -1
    ratios = [difflib.SequenceMatcher(a=src, b=c).ratio() for c in cands_norm]
    return max(range(len(cands_norm)), key=lambda i: ratios[i])

def build_statement_map(tableau_sql: str, fabric_sql: str) -> List[Tuple[Tuple[int,int], Tuple[int,int]]]:
    """
    Map Tableau statements to Fabric statements using normalized fuzzy matching.
    Returns list of ((src_start, src_end), (tgt_start, tgt_end)).
    """
    src_stmts = _split_statements(tableau_sql)
    tgt_stmts = _split_statements(fabric_sql)
    tgt_norm = [_normalize(s['text']) for s in tgt_stmts]

    mapping = []
    for s in src_stmts:
        idx = _best_match(_normalize(s['text']), tgt_norm)
        if idx != -1:
            tgt_span = (tgt_stmts[idx]['start'], tgt_stmts[idx]['end'])
        else:
            tgt_span = (None, None)
        mapping.append(((s['start'], s['end']), tgt_span))
    return mapping

def _coerce_severity(val: Any) -> str:
    s = str(val or '').lower()
    return s if s in {'info','warning','error'} else 'warning'

def simplify_flags(tableau_sql: str, fabric_sql: str, raw_flags: Optional[List]=None) -> List[SimpleFlag]:
    """
    Convert arbitrary 'raw_flags' into SimpleFlag objects with line ranges.
    This function is defensive: any unexpected flag format becomes a safe 'warning' flag.
    """
    mapping = build_statement_map(tableau_sql, fabric_sql)

    # If no flags, emit a friendly info flag (optional, can be omitted)
    if not raw_flags:
        return [SimpleFlag('info', 'No issues detected by converter.', None, None)]

    out: List[SimpleFlag] = []
    for rf in raw_flags:
        try:
            # Dictionary form: {"message":..., "severity":..., "statement_index":..., "source_span":..., "target_span":...}
            if isinstance(rf, dict):
                msg = str(rf.get('message', 'Unspecified issue'))
                sev = _coerce_severity(rf.get('severity'))
                code = rf.get('code')
                hint = rf.get('hint')

                src_span = rf.get('source_span')
                tgt_span = rf.get('target_span')
                idx = rf.get('statement_index')

                if (not isinstance(src_span, tuple) or not isinstance(tgt_span, tuple)) and isinstance(idx, int):
                    if 0 <= idx < len(mapping):
                        src_span, tgt_span = mapping[idx]

                out.append(SimpleFlag(sev, msg, src_span if isinstance(src_span, tuple) else None,
                                      tgt_span if isinstance(tgt_span, tuple) else None,
                                      code=code, hint=hint))

            else:
                # Tuple form: (message, severity, statement_index)
                msg = str(rf[0]) if len(rf) > 0 else 'Unspecified issue'
                sev = _coerce_severity(rf[1] if len(rf) > 1 else 'warning')
                idx = rf[2] if len(rf) > 2 else None

                src_span, tgt_span = (None, None)
                if isinstance(idx, int) and 0 <= idx < len(mapping):
                    src_span, tgt_span = mapping[idx]

                out.append(SimpleFlag(sev, msg, src_span, tgt_span))

        except Exception:
                       # Never leak exceptions to UI
            out.append(SimpleFlag('warning',
                                  'A flag could not be normalized due to unexpected format.',
                                  None, None,
                                  code='FLAG_PARSE',
                                  hint='Please report the original flag payload.'))


In [9]:

# Register in-notebook flagging code as a module named "flagging_engine"
import types, sys

flagging_engine = types.ModuleType("flagging_engine")
# Expose the classes/functions you defined in Cell 3# Expose the classes/functions you defined in Cell 3
flagging_engine.SimpleFlag = SimpleFlag
flagging_engine.simplify_flags = simplify_flags
flagging_engine.build_statement_map = build_statement_map


In [10]:

# Separate handlers class to handle operations on ApplicationController
import os
from flagging_engine import simplify_flags  # <-- NEW

class handlers:
    def __init__(self, controller):
        self.controller = controller

    def browse_file(self):
        # Open dialog to select SQL file (Allow .txt for Tableau copy/paste cases)
        file_path = filedialog.askopenfilename(
            title="Select Tableau SQL File",
            filetypes=[
                ("SQL Files", "*.sql"),
                ("Text Files", "*.txt"),
                ("All Files", "*.*")
            ]
        )

        if file_path:
            self.controller.current_file_path = file_path
            self.controller.ui.set_file_path(file_path)

            # Provide file info, to confirm the file selected
            file_info = self.controller.file_handler.get_file_info(file_path)
            if file_info:
                info_text = f"File: {file_info['filename']} | Size: {file_info['size_kb']:.2f} KB"
                self.controller.ui.set_file_info(info_text)
                self.controller.ui.set_status(f"File selected: {file_info['filename']}")
            else:
                # Fallback if get_file_info returned None
                self.controller.ui.set_status(f"File selected: {os.path.basename(file_path)}")

    def load_and_convert(self):
        '''
        Load and convert the selected file
        '''
        # Check if a valid file is selected
        if not self.controller.current_file_path:
            self.controller.ui.show_warning("No File", "Please select a SQL file first.")
            return

        # Validate file
        is_valid, error = self.controller.file_handler.validate_file(self.controller.current_file_path)
        if not is_valid:
            self.controller.ui.show_error("Invalid File", error)
            return

        # Use try for any errors not considered (lots of commands and new releases)
        try:
            # Read file using file_handler.py
            raw_sql = self.controller.file_handler.read_file(self.controller.current_file_path)

            # Clean and prepare SQL using data_cleaner.py
            prepared_data = self.controller.cleaner.prepare_for_parsing(raw_sql)
            self.controller.tableau_sql = prepared_data['cleaned_query']

            # Update UI with Tableau SQL (User's Input)
            self.controller.ui.set_tableau_sql(self.controller.tableau_sql)

            # Convert SQL using sql_parser.py
            self.controller.fabric_sql, self.controller.current_metrics, self.controller.flagged_items = \
                self.controller.converter.convert_query(self.controller.tableau_sql)

            # Display Fabric SQL
            self.controller.ui.set_fabric_sql(self.controller.fabric_sql)

            # --- NEW: Simplify flags so UI is robust to line wrapping differences ---
            simple_flags = simplify_flags(
                self.controller.tableau_sql,
                self.controller.fabric_sql,
                self.controller.flagged_items
            )
            self.controller.flagged_items = simple_flags

            # Display flagged items in UI
            self.controller.ui.display_flagged_items(self.controller.flagged_items, self.controller.current_metrics)

            # Enable save button
            self.controller.ui.enable_save_button()

            # Always define status_text to avoid UnboundLocalError
            status_text = "Conversion complete!"
            # Count only warnings/errors
            flag_count = sum(1 for f in self.controller.flagged_items if f.severity in {'warning', 'error'})
            if flag_count:
                status_text += f" | {flag_count} item(s) flagged for review"

            self.controller.ui.set_status(status_text)

            # Show completion message
            self.controller.ui.show_info("Success", "SQL conversion completed successfully!")

        except Exception as e:
            self.controller.ui.show_error(
                "There was an issue with conversion",
                f"An error occurred during conversion:\n{str(e)}"
            )

    def save_converted_sql(self):
        """Save the converted Fabric SQL to a file."""
        current_fabric_sql = self.controller.ui.get_fabric_sql()

        # If file was unable to convert {incorrect file type or syntax, or just empty file}
        if not current_fabric_sql:
            self.controller.ui.show_warning("No Data", "No converted SQL to save.")
            return

        # If file was able to convert, then save it
        if self.controller.current_file_path:
            default_path = self.controller.file_handler.generate_output_filename(self.controller.current_file_path)
        else:
            default_path = "converted_fabric.sql"

        save_path = filedialog.asksaveasfilename(
            title="Save SQL Generated File",
            initialfile=default_path,
            defaultextension=".sql",
            filetypes=[("SQL Files", "*.sql"), ("Text Files", "*.txt"), ("All Files", "*.*")]
        )

        if save_path:
            try:
                self.controller.file_handler.write_file(save_path, current_fabric_sql)
                self.controller.ui.show_info("Success", f"Converted SQL saved to:\n{save_path}")
                self.controller.ui.set_status(f"File saved successfully: {save_path}")
            except Exception as e:
                self.controller.ui.show_error("Save Error", f"Failed to save file:\n{str(e)}")

    def clear_all(self):
        """Reset application"""
        self.controller.current_file_path = None
        self.controller.tableau_sql = ""
        self.controller.fabric_sql = ""
        self.controller.current_metrics = None
        self.controller.flagged_items = []
        self.controller.ui.clear_all()

ModuleNotFoundError: No module named 'flagging_engine'

In [18]:
class ApplicationController:
    def __init__(self):
        self.root = tk.Tk()
        
        # Initialize logic components
        self.converter = SQLConverter()
        self.cleaner = SQLCleaner()
        self.file_handler = FileHandler()
        
        # App state
        self.current_file_path = None
        self.tableau_sql = ""
        self.fabric_sql = ""
        self.current_metrics = None
        self.flagged_items = []
        
        # Initialize handlers, passing SELF (this controller instance)
        self.handlers = handlers(self)
        
        # Callbacks point to the handler instance's methods
        callbacks = {
            'on_browse': self.handlers.browse_file,
            'on_convert': self.handlers.load_and_convert,
            'on_save': self.handlers.save_converted_sql,
            'on_clear': self.handlers.clear_all
        }
        self.ui = SQLConverterUI(self.root, callbacks)

    def run(self):
        try:
            self.ui.run()
        except Exception as e:
            print(f"Application error: {e}")


In [19]:
def main():
    app = ApplicationController()
    app.run()

if __name__ == "__main__":
    main()